<a href="https://colab.research.google.com/github/adnan855570/Natural_language_Processing/blob/main/Byte_Pair_Encoding_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Authenticate and authorize access to Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

file_path = '/content/drive/MyDrive/FYP/Threatening_Dataset_R18.xlsx'

df = pd.read_excel(file_path)

print(df.head)



<bound method NDFrame.head of                                                  Tweets  label  S/G
0                                          بکواس مت کرو      1    1
1     تمہاری ہیجڑا فورس ایک نہتے کے سامنے بکری بنی ہ...      1    0
2     آفیسر سمیتبھارتی فوجی جہنم واصل،بنکرز تباہ بھا...      1    1
3                    غدار منافق میر اللہ تمہیں زلیل کرے      1    1
4     اگست پر آپ بھارت کو کیا پیغام دینا چاہیں گے؟ م...      1    0
...                                                 ...    ...  ...
3559          ارے شعر ہے حقیقت میں کہیں پیچھے نا پڑنا ۔      0    2
3560  ہندوستانی باوردی غنڈا ایل او سی پر پتھر مار کے...      0    2
3561                                یہ خبر افسوس ناک ھے      0    2
3562  جناب محترم سلمان وفادارصاھب کیا پارلیمنٹ آئینی...      0    2
3563  چھوٹے قد کی لڑکی اور شہد کی مکھی دونوں ہی بہت ...      0    2

[3564 rows x 3 columns]>


In [4]:
from collections import defaultdict, Counter

# Sample dataset
tweets = df['Tweets']
vocab = None
def get_vocab(tweets):
    """Extracts vocabulary and counts from tweets."""
    vocab = Counter()
    for tweet in tweets:
        words = tweet.strip().split()
        for word in words:
            # Splitting words into characters and adding end of word token
          vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab

def get_stats(vocab):
    """Computes frequencies of pairs of symbols."""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, vocab_in):
    """Merges the most frequent pair in the vocabulary."""
    vocab_out = {}
    bigram = ' '.join(pair)
    replacer = ''.join(pair)
    for word in vocab_in:
        w_out = word.replace(bigram, replacer)
        vocab_out[w_out] = vocab_in[word]
    return vocab_out

def train_bpe(tweets, num_merges=100):
    """Trains a BPE model."""
    vocab = get_vocab(tweets)
    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)
    return vocab

# Train BPE on the given dataset
num_merges = 10  # Number of merges, adjust as needed
bpe_vocab = train_bpe(tweets, num_merges)

# Print the resulting vocabulary
for i, (token, freq) in enumerate(bpe_vocab.items()):
    print(f"Token: {token} Frequency: {freq}")
    if i >= 20:  # Limiting output for brevity
        break



Token: ب ک و ا س </w> Frequency: 161
Token: م ت</w> Frequency: 24
Token: ک ر و</w> Frequency: 93
Token: ت م ہ ا ر ی</w> Frequency: 32
Token: ہ ی ج ڑ ا</w> Frequency: 3
Token: ف و ر س </w> Frequency: 2
Token: ا ی ک </w> Frequency: 291
Token: ن ہ ت ے</w> Frequency: 5
Token: ک ے</w> Frequency: 1629
Token: س ا م ن ے</w> Frequency: 32
Token: ب ک ر ی</w> Frequency: 1
Token: ب ن ی</w> Frequency: 8
Token: ہ ے</w> Frequency: 1285
Token: ڈ و ب </w> Frequency: 29
Token: م ر و</w> Frequency: 33
Token: آ ف ی س ر</w> Frequency: 6
Token: س م ی ت ب ھ ا ر ت ی</w> Frequency: 2
Token: ف و ج ی</w> Frequency: 40
Token: ج ہ ن م </w> Frequency: 39
Token: و ا ص ل ، ب ن ک ر ز </w> Frequency: 1
Token: ت ب ا ہ</w> Frequency: 58


In [5]:
# install and import libraries
from collections import Counter, defaultdict
from transformers import AutoTokenizer

class BPE():
    """Byte-Pair Encoding: Subword-based tokenization algorithm."""

    def __init__(self, corpus, vocab_size):
        """Initialize BPE tokenizer."""
        self.corpus = corpus
        self.vocab_size = vocab_size

        # pre-tokenize the corpus into words, BERT pre-tokenizer is used here
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.word_freqs = defaultdict(int)
        self.splits = {}
        self.merges = {}


    def train(self):
        """Train BPE tokenizer."""

        # compute the frequencies of each word in the corpus
        for text in self.corpus:
            words_with_offsets = self.tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
            new_words = [word for word, offset in words_with_offsets]
            for word in new_words:
                self.word_freqs[word] += 1

        # compute the base vocabulary of all characters in the corpus
        alphabet = []
        for word in self.word_freqs.keys():
            for letter in word:
                if letter not in alphabet:
                    alphabet.append(letter)
        alphabet.sort()

        # add the special token </w> at the beginning of the vocabulary
        vocab = ["</w>"] + alphabet.copy()

        # split each word into individual characters before training
        self.splits = {word: [c for c in word] for word in self.word_freqs.keys()}

        # merge the most frequent pair iteratively until the vocabulary size is reached
        while len(vocab) < self.vocab_size:

            # compute the frequency of each pair
            pair_freqs = self.compute_pair_freqs()

            # find the most frequent pair
            best_pair = ""
            max_freq = None
            for pair, freq in pair_freqs.items():
                if max_freq is None or max_freq < freq:
                    best_pair = pair
                    max_freq = freq

            # merge the most frequent pair
            self.splits = self.merge_pair(*best_pair)
            self.merges[best_pair] = best_pair[0] + best_pair[1]
            vocab.append(best_pair[0] + best_pair[1])
        return self.merges


    def compute_pair_freqs(self):
        """Compute the frequency of each pair."""

        pair_freqs = defaultdict(int)
        for word, freq in self.word_freqs.items():
            split = self.splits[word]
            if len(split) == 1:
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] += freq
        return pair_freqs


    def merge_pair(self, a, b):
        """Merge the given pair."""

        for word in self.word_freqs:
            split = self.splits[word]
            if len(split) == 1:
                continue
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    split = split[:i] + [a + b] + split[i + 2 :]
                else:
                    i += 1
            self.splits[word] = split
        return self.splits


    def tokenize(self, text):
        """Tokenize a given text with trained BPE tokenizer (including pre-tokenization, split, and merge)."""

        pre_tokenize_result = self.tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
        pre_tokenized_text = [word for word, offset in pre_tokenize_result]
        splits_text = [[l for l in word] for word in pre_tokenized_text]

        for pair, merge in self.merges.items():
            for idx, split in enumerate(splits_text):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2 :]
                    else:
                        i += 1
                splits_text[idx] = split
        result = sum(splits_text, [])
        return result

In [6]:
df['Tweets']

0                                            بکواس مت کرو
1       تمہاری ہیجڑا فورس ایک نہتے کے سامنے بکری بنی ہ...
2       آفیسر سمیتبھارتی فوجی جہنم واصل،بنکرز تباہ بھا...
3                      غدار منافق میر اللہ تمہیں زلیل کرے
4       اگست پر آپ بھارت کو کیا پیغام دینا چاہیں گے؟ م...
                              ...                        
3559            ارے شعر ہے حقیقت میں کہیں پیچھے نا پڑنا ۔
3560    ہندوستانی باوردی غنڈا ایل او سی پر پتھر مار کے...
3561                                  یہ خبر افسوس ناک ھے
3562    جناب محترم سلمان وفادارصاھب کیا پارلیمنٹ آئینی...
3563    چھوٹے قد کی لڑکی اور شہد کی مکھی دونوں ہی بہت ...
Name: Tweets, Length: 3564, dtype: object

In [10]:
# import the Wikipedia corpus used for training

# set the hyperparameter of vocabulary size
vocab_size = 1000

# create a BPE tokenizer object
MyBPE = BPE(corpus=df['Tweets'], vocab_size=vocab_size)

# train BPE tokenizer with Wikipedia corpus
MyBPE.train()

# tokenize the given text
text = df['Tweets']
print(f"\nBPE tokenization result of text\n'{text}'")
# print(MyBPE.tokenize(text))
# Iterate over the Series and apply tokenization to each text entry
# tokenized_texts = df['Tweets'].apply(lambda text: MyBPE.tokenize(text))

# Now 'tokenized_texts' contains the tokenized version of each entry in 'df['Tweets']'
# print(tokenized_texts)


for index, (original_text, tokenized_text) in enumerate(zip(df['Tweets'], tokenized_texts)):
    print(f"Tweet #{index}: {original_text}")
    print(f"Tokenized: {tokenized_text}\n")

    # Optionally, add a break or condition to limit the number of outputs for readability
    if index == 3000:  # Adjust the number to display more or less
        break

Streaming output truncated to the last 5000 lines.
Tokenized: ['د', 'س', 'سال', 'بعد', 'نہ', 'رہے', 'گا', 'ب', 'انس', 'نہ', 'بج', 'ے', 'گی', 'ب', 'انس', 'ری', 'سمجھ', 'نے', 'والے', 'سمجھ', 'گے', 'جو', 'نہ', 'سمج', 'ھے', 'وہ', 'بان', 'د', 'ری']

Tweet #1335: پاکستان کو آنکھیں دکھانے والے بھارت کی اقتصادی شہ رگ اسلامی ممالک کے ہاتھ لیکن دراصل وہ بولنے سے کیوں قاصر ہیں؟ و…
Tokenized: ['پاکستان', 'کو', 'آنکھ', 'یں', 'د', 'کھ', 'انے', 'والے', 'بھارت', 'کی', 'ا', 'قت', 'ص', 'ادی', 'شہ', 'رگ', 'اسلام', 'ی', 'م', 'مال', 'ک', 'کے', 'ہاتھ', 'لیکن', 'در', 'اصل', 'وہ', 'بول', 'نے', 'سے', 'کیوں', 'ق', 'ا', 'صر', 'ہیں', '؟', 'و', '…']

Tweet #1336: دشمنوں تم نے کس قوم کو للکارا
Tokenized: ['دشمن', 'وں', 'تم', 'نے', 'کس', 'قوم', 'کو', 'للکارا']

Tweet #1337: ابھی ایک جیالہ ایک ریٹائرڈ جنرل کو کہتا ہے کہ  ابھی میں آپ کے ہاتھ ہتھیار لے لوں آپ مجھ سے ڈریں گے
Tokenized: ['ابھی', 'ایک', 'ج', 'یا', 'لہ', 'ایک', 'ری', 'ٹ', 'ائ', 'ر', 'ڈ', 'جنرل', 'کو', 'کہ', 'تا', 'ہے', 'کہ', 'ابھی', 'میں', 'آپ', 'کے', 'ہات